## Import libs

In [ ]:
# Standard imports used by the notebook
import os
import json
import subprocess
import requests
import time
from pathlib import Path

def get_access_token():
    """Return an access token using gcloud.
    Raises a RuntimeError if gcloud is not available or not authenticated.
    """
    try:
        out = subprocess.check_output(["gcloud", "auth", "print-access-token"])
        return out.decode().strip()
    except Exception as e:
        raise RuntimeError("Failed to get access token. Make sure gcloud is installed and you are authenticated (gcloud auth login).") from e

print("Imports and helper ready.")

## Load data

In [ ]:
# Path to the JSONL file shipped with the repo
DATA_PATH = Path("Milestone-5/math-agent/combined_benchmarks.jsonl")
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Data file not found at {DATA_PATH}. Run from repository root or adjust the path.")

data = []
with DATA_PATH.open("r") as fh:
    for ln in fh:
        ln = ln.strip()
        if not ln:
            continue
        data.append(json.loads(ln))

print(f"Loaded {len(data)} records from {DATA_PATH}")

## Set env variables

In [ ]:
# These should be set in your environment or you can replace the placeholders here
ENDPOINT_URL = os.environ.get("ENDPOINT_URL") or "ENDPOINT_URL_GOES_HERE"
PROJECT_ID = os.environ.get("PROJECT_ID") or "PROJECT_ID_GOES_HERE"
LOCATION = os.environ.get("LOCATION") or "LOCATION_GOES_HERE"
ENDPOINT_ID = os.environ.get("ENDPOINT_ID") or "ENDPOINT_ID_GOES_HERE"

# Quick sanity check - replace placeholders with real values before running requests
print("ENDPOINT_URL:", ENDPOINT_URL)
print("PROJECT_ID:", PROJECT_ID)
print("LOCATION:", LOCATION)
print("ENDPOINT_ID:", ENDPOINT_ID)

## Call one single row from jsonl 

In [ ]:
# Pick a single record (index 0) and call the Vertex AI endpoint using the same payload shape as your curl example
idx = 0
record = data[idx]
# extract the last user message content if available
messages = record.get("body", {}).get("messages", [])
user_prompt = None
for m in reversed(messages):
    if m.get("role") == "user":
        user_prompt = m.get("content")
        break

if not user_prompt:
    raise ValueError("No user prompt found for this record")

payload = {
    "instances": [
        {
            "prompt": user_prompt,
            "temperature": 0,
        }
    ]
}

print("Calling endpoint with prompt:")
print(user_prompt)

# Build the URL and headers, then perform the request
token = get_access_token()
url = f"https://{ENDPOINT_URL}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:predict"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
resp = requests.post(url, headers=headers, json=payload, timeout=60)
print("HTTP status:", resp.status_code)
# If the service returns JSON, pretty-print it
try:
    print(json.dumps(resp.json(), indent=2))
except Exception:
    print(resp.text)

## call all rows in a loop and save it, also checkpoint it

In [ ]:
# Iterate through all records, call the endpoint, and save responses to a JSONL file.
OUT_PATH = Path(f"Milestone-5/math-agent/{ENDPOINT_ID}_predictions.jsonl")
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

with OUT_PATH.open("w") as outf:
    for i, record in enumerate(data):
        messages = record.get("body", {}).get("messages", [])
        user_prompt = None
        for m in reversed(messages):
            if m.get("role") == "user":
                user_prompt = m.get("content")
                break
        if not user_prompt:
            # skip records that don't have a user prompt
            continue

        payload = {"instances": [{"prompt": user_prompt, "temperature": 0}]}
        try:
            token = get_access_token()
            resp = requests.post(
                f"https://{ENDPOINT_URL}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:predict",
                headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
                json=payload,
                timeout=60,
            )
            try:
                body = resp.json()
            except Exception:
                body = resp.text
            out = {
                "index": i,
                "status": resp.status_code,
                "prompt": user_prompt,
                "response": body,
            }
        except Exception as e:
            out = {"index": i, "error": str(e), "prompt": user_prompt}

        outf.write(json.dumps(out) + "")
        # checkpoint/flush periodically
        if (i + 1) % 10 == 0:
            outf.flush()
        # small delay to avoid rate limits
        time.sleep(0.5)

print("Finished. Responses saved to:", OUT_PATH)